<a href="https://colab.research.google.com/github/atnawshin/Machine-Learning/blob/main/BanglaDataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import re
import string
from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
df = pd.read_excel('fbdataset.xlsx')
df.head(2)

,@LABEL,Created Time,Writer,Reaction Count,Comment Count,Share Count,Label,Text
0,163059227060505_6066791063353929,2022-07-21 04:55:09,Prothom Alo,10,0,0,1,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...
1,163059227060505_6066743890025313,2022-07-21 04:30:30,Prothom Alo,155,15,1,0,- Dear বান্ধুবি আর কতো অপে'ক্ষা করাবি বইন.?🙄 ...


In [ ]:
print(df)

                                @LABEL         Created Time       Writer  \
0     163059227060505_6066791063353929  2022-07-21 04:55:09  Prothom Alo   
1     163059227060505_6066743890025313  2022-07-21 04:30:30  Prothom Alo   
2     163059227060505_6066725910027111  2022-07-21 04:21:08  Prothom Alo   
3     163059227060505_6066640633368972  2022-07-21 03:40:12  Prothom Alo   
4     163059227060505_6066608523372183  2022-07-21 03:25:09  Prothom Alo   
..                                 ...                  ...          ...   
278  103039661944184_366206938960787_2  2022-07-15 13:30:58         None   
279  103039661944184_366206938960787_3  2022-07-15 21:06:31         None   
280  103039661944184_366206548960826_1  2022-07-15 13:01:39         None   
281  103039661944184_365828448998636_1  2022-07-15 02:22:55         None   
282  103039661944184_365827455665402_1  2022-07-15 01:17:21         None   

     Reaction Count  Comment Count Share Count  Label  \
0                10           

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   @LABEL          283 non-null    object
 1   Created Time    283 non-null    object
 2   Writer          283 non-null    object
 3   Reaction Count  283 non-null    int64 
 4   Comment Count   283 non-null    int64 
 5   Share Count     283 non-null    object
 6   Label           283 non-null    int64 
 7   Text            283 non-null    object
dtypes: int64(3), object(5)
memory usage: 17.8+ KB


In [ ]:
df.shape

(283, 8)

In [ ]:
df.dtypes

@LABEL            object
Created Time      object
Writer            object
Reaction Count     int64
Comment Count      int64
Share Count       object
Label              int64
Text              object
dtype: object

In [ ]:
data = df[['Text', 'Label']]
data.head()

,Text,Label
0,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...,1
1,- Dear বান্ধুবি আর কতো অপে'ক্ষা করাবি বইন.?🙄 ...,0
2,- Story Of My Life😔 - দেখতে বাজে😪 - কেউ পাত্ত...,1
3,- আচ্ছা! - আমার কাছে যদি চাওয়ার কিছু সুযোগ পা...,0
4,"- আজ পর্যন্ত কারোর😓 - Dp, Cover, Status, Day,...",0


In [ ]:
#columns rename
#data = dta.rename(columns={"Recommended IND": "Recommended", "review": "Review"})
#data.head()
data.isnull().sum()

Text                    0
Label                   0
Text1                   0
Removed Puncutation     0
Removed Stopped word    0
Translated Text         0
dtype: int64

In [ ]:
#drop null values
data.dropna(subset=['Text'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
data.isnull().sum()

Text     0
Label    0
dtype: int64

In [ ]:
data['Text'] = data['Text'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data.shape

(283, 2)

In [ ]:
data['Text'].value_counts()

চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়িত ব্যক্তিদের শাস্তি নিশ্চিত করার ঘোষণা দিয়েছেন চট্টগ্রাম বিশ্ববিদ্যালয়ের ভারপ্রাপ্ত রেজিস্ট্রার এস এম মনিরুল হাসান।    1
thik                                                                                                                                                                   1
Orpita Saha 🙄🙄🙄                                                                                                                                                        1
😁😁😁                                                                                                                                                                    1
Aho vatija aho tmre dimu na 😏😒😒                                                                                                                                        1
                                                                                                                                                           

In [ ]:
data.duplicated().sum()

0

Translat all the texts into Bangla

In [ ]:
#!pip3 install googletrans==3.1.0a0

In [ ]:
#Translate the other languages to english
import googletrans
from googletrans import *
translator = googletrans.Translator()

data['Text'] = data['Text'].astype(str) #changing datatype to string
data['Text1'] = data['Text'].apply(translator.translate, src='auto', dest='bn').apply(getattr, args=('text',))
data

,Text,Label,Text1,Removed Puncutation,Removed Stopped word,Translated Text
0,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...,1,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...,"[দিনের, শিক্ষার্থীকে, যৌন, নিপীড়নে, জড়িত, ব্...",চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...
1,- Dear বান্ধুবি আর কতো অপে'ক্ষা করাবি বইন.?🙄 ...,0,- Dear বান্ধুবি আর কতো অপে'ক্ষা করাবি বইন.?🙄 ...,Dear বান্ধুবি আর কতো অপেক্ষা করাবি বইন🙄 এইব...,"[Dear, বান্ধুবি, কতো, অপেক্ষা, করাবি, বইন🙄, এই...",- Dear বান্ধুবি আর কতো অপে'ক্ষা করাবি বইন.?🙄 ...
2,- Story Of My Life😔 - দেখতে বাজে😪 - কেউ পাত্ত...,1,- Story Of My Life😔 - দেখতে বাজে😪 - কেউ পাত্ত...,Story Of My Life😔 দেখতে বাজে😪 কেউ পাত্তা দ...,"[Story, Of, My, Life😔, বাজে😪, পাত্তা, 😒, ভালোব...",- Story Of My Life😔 - দেখতে বাজে😪 - কেউ পাত্ত...
3,- আচ্ছা! - আমার কাছে যদি চাওয়ার কিছু সুযোগ পা...,0,- আচ্ছা! - আমার কাছে যদি চাওয়ার কিছু সুযোগ পা...,আচ্ছা আমার কাছে যদি চাওয়ার কিছু সুযোগ পাও😱 ...,"[আচ্ছা, চাওয়ার, সুযোগ, পাও😱, চাইবে🙄🐸]",- আচ্ছা! - আমার কাছে যদি চাওয়ার কিছু সুযোগ পা...
4,"- আজ পর্যন্ত কারোর😓 - Dp, Cover, Status, Day,...",0,"- আজ পর্যন্ত কারোর😓 - Dp, Cover, Status, Day,...",আজ পর্যন্ত কারোর😓 Dp Cover Status Day Bio W...,"[কারোর😓, Dp, Cover, Status, Day, Bio, Wallpape...","- আজ পর্যন্ত কারোর😓 - Dp, Cover, Status, Day,..."
...,...,...,...,...,...,...
278,Right,0,ঠিক,ঠিক,[],ঠিক
279,Ak dom,0,ঘর হলে,ঘর হলে,[ঘর],ঘর হলে
280,Bal......,0,বল......,বল,[বল],বল......
281,কিছু কইলাম না 😈,0,কিছু কইলাম না 😈,কিছু কইলাম না 😈,"[কইলাম, 😈]",কিছু কইলাম না 😈


In [ ]:
data.head(2)

Removing Puntuation

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punctuation(Text1):
    punctuation_list = str(string.punctuation) + "।"
    text_nopunct = "".join([c for c in Text1 if c not in punctuation_list])
    return text_nopunct

In [ ]:
def remove_garbage(a):
  a= "".join(i for i in a if i in ["।"] or 2432 <= ord(i) <= 2559 or ord(i)== 32)
  removed=" ".join(a.split())
  return removed

In [ ]:
data['Removed Puncutation'] = data['Text1'].apply(lambda x:remove_punctuation(x))
data.head(2)

,Text,Label,Text1,Removed Puncutation
0,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...,1,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...
1,- Dear বান্ধুবি আর কতো অপে'ক্ষা করাবি বইন.?🙄 ...,0,- Dear বান্ধুবি আর কতো অপে'ক্ষা করাবি বইন.?🙄 ...,Dear বান্ধুবি আর কতো অপেক্ষা করাবি বইন🙄 এইব...


Remove English

In [ ]:
#data['remove Eng'] = data['Removed Puncutation'].apply(lambda x:remove_garbage(x))
#data

In [ ]:
!pip install bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 44.8 MB/s 
     |████████████████████████████████| 23.9 MB 1.5 MB/s 
     |████████████████████████████████| 965 kB 54.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from bnlp.corpus import stopwords
from bnlp.corpus.util import remove_stopwords

data['Removed Stopped word'] = data['Removed Puncutation'].apply(lambda x: remove_stopwords(x,stopwords))
data

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


,Text,Label,Text1,Removed Puncutation,Removed Stopped word
0,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...,1,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...,চার দিনের মধ্যে শিক্ষার্থীকে যৌন নিপীড়নে জড়ি...,"[দিনের, শিক্ষার্থীকে, যৌন, নিপীড়নে, জড়িত, ব্..."
1,- Dear বান্ধুবি আর কতো অপে'ক্ষা করাবি বইন.?🙄 ...,0,- Dear বান্ধুবি আর কতো অপে'ক্ষা করাবি বইন.?🙄 ...,Dear বান্ধুবি আর কতো অপেক্ষা করাবি বইন🙄 এইব...,"[Dear, বান্ধুবি, কতো, অপেক্ষা, করাবি, বইন🙄, এই..."
2,- Story Of My Life😔 - দেখতে বাজে😪 - কেউ পাত্ত...,1,- Story Of My Life😔 - দেখতে বাজে😪 - কেউ পাত্ত...,Story Of My Life😔 দেখতে বাজে😪 কেউ পাত্তা দ...,"[Story, Of, My, Life😔, বাজে😪, পাত্তা, 😒, ভালোব..."
3,- আচ্ছা! - আমার কাছে যদি চাওয়ার কিছু সুযোগ পা...,0,- আচ্ছা! - আমার কাছে যদি চাওয়ার কিছু সুযোগ পা...,আচ্ছা আমার কাছে যদি চাওয়ার কিছু সুযোগ পাও😱 ...,"[আচ্ছা, চাওয়ার, সুযোগ, পাও😱, চাইবে🙄🐸]"
4,"- আজ পর্যন্ত কারোর😓 - Dp, Cover, Status, Day,...",0,"- আজ পর্যন্ত কারোর😓 - Dp, Cover, Status, Day,...",আজ পর্যন্ত কারোর😓 Dp Cover Status Day Bio W...,"[কারোর😓, Dp, Cover, Status, Day, Bio, Wallpape..."
...,...,...,...,...,...
278,Right,0,ঠিক,ঠিক,[]
279,Ak dom,0,ঘর হলে,ঘর হলে,[ঘর]
280,Bal......,0,বল......,বল,[বল]
281,কিছু কইলাম না 😈,0,কিছু কইলাম না 😈,কিছু কইলাম না 😈,"[কইলাম, 😈]"
